In [ ]:
import requests
from requests.auth import HTTPBasicAuth
import json
from nltk.text import sent_tokenize
from nltk import word_tokenize
from rank_bm25 import BM25Okapi
from rank_bm25 import BM25Plus
from rank_bm25 import BM25L
from utils import create_dataset

# disable warnings related to verify
requests.urllib3.disable_warnings()

# download requried resources
# nltk.download('punkt', download_dir='./')

## Document API

In [ ]:
url = "https://guacamole.univ-avignon.fr/dblp1/_search?q=%22Digital%20assistant%22&size=10"
resp = requests.get(url, auth=HTTPBasicAuth('inex', 'qatc2011'), verify=False)

In [ ]:
contents = json.loads(resp.content)

In [ ]:
contents['hits']['hits'][0]['_source'].keys()

In [ ]:
docs = [hit['_source'] for hit in contents['hits']['hits']]

In [ ]:
docs[0]

In [ ]:
sent_tokenize(docs[0]['abstract'])

## BM25

In [ ]:
dataset = create_dataset('../topics/SP12022topics.json','../topics/topic_related_content/')
# with open('dataset.json') as f_in:
#     dataset = json.load(f_in)

In [ ]:
with open('../topics/SP12022topics.json') as f_in:
    topics = json.load(f_in)

In [ ]:
topics[-1]

In [ ]:
data = dataset[0]
docs = data['docs']
tokenized_corpus = [word_tokenize(doc.lower()) for doc in docs]
bm25 = BM25Okapi(tokenized_corpus)
bm25plus = BM25Plus(tokenized_corpus)
bm25L = BM25L(tokenized_corpus)

In [ ]:
topic_text = data['topic_text']
title_as_query_scores1 = bm25.get_scores(word_tokenize(topic_text.lower()))
print(title_as_query_scores1)
title_as_query_scores2 = bm25plus.get_scores(word_tokenize(topic_text.lower()))
print(title_as_query_scores2)
title_as_query_scores3 = bm25L.get_scores(word_tokenize(topic_text.lower()))
print(title_as_query_scores3)

In [ ]:
topic_content = data['topic_content']
article_as_query_scores1 = bm25.get_scores(topic_content)
print(article_as_query_scores1)
article_as_query_scores2 = bm25plus.get_scores(topic_content)
print(article_as_query_scores2)
article_as_query_scores3 = bm25L.get_scores(topic_content)
print(article_as_query_scores3)

In [ ]:
print('============== BM25Okapi ==============')
print('============== Query title ==============')
print(topic_text)
print('============== Highest ranked passage by title ==============')
print(docs[title_as_query_scores1.argmax()])
print('============== Highest ranked passage by content ==============')
print(docs[article_as_query_scores1.argmax()])

print('============== BM25plus ==============')
print('============== Query title ==============')
print(topic_text)
print('============== Highest ranked passage by title ==============')
print(docs[title_as_query_scores2.argmax()])
print('============== Highest ranked passage by content ==============')
print(docs[article_as_query_scores2.argmax()])

print('============== BM25OL==============')
print('============== Query title ==============')
print(topic_text)
print('============== Highest ranked passage by title ==============')
print(docs[title_as_query_scores3.argmax()])
print('============== Highest ranked passage by content ==============')
print(docs[article_as_query_scores3.argmax()])
